# Data Cleaning/Preproccessing

In [1788]:
import pandas as pd

In [1789]:
seasons = pd.read_csv('Player Totals.csv') # season by season data on every NBA player since 1947 from Sumitro Datta on Kaggle

In [1790]:
pd.set_option('display.max_columns', None)

In [1791]:
seasons.head() 

,seas_id,season,player_id,player,birth_year,pos,age,experience,lg,tm,g,gs,mp,fg,fga,fg_percent,x3p,x3pa,x3p_percent,x2p,x2pa,x2p_percent,e_fg_percent,ft,fta,ft_percent,orb,drb,trb,ast,stl,blk,tov,pf,pts
0,30458,2023,5025,A.J. Green,NaN,SG,23.0,1,NBA,MIL,35,1.0,345.0,53,125,0.424,44.0,105.0,0.419,9,20,0.450,0.600,4,4,1.000,6.0,39.0,45.0,22,6.0,0.0,9.0,31,154
1,30459,2023,5026,A.J. Lawson,NaN,SG,22.0,1,NBA,TOT,15,0.0,108.0,22,44,0.500,10.0,25.0,0.400,12,19,0.632,0.614,2,8,0.250,6.0,15.0,21.0,2,2.0,0.0,3.0,11,56
2,30460,2023,5026,A.J. Lawson,NaN,SG,22.0,1,NBA,MIN,1,0.0,2.0,1,1,1.000,0.0,0.0,NaN,1,1,1.000,1.000,0,0,NaN,0.0,1.0,1.0,0,0.0,0.0,0.0,1,2
3,30461,2023,5026,A.J. Lawson,NaN,SG,22.0,1,NBA,DAL,14,0.0,106.0,21,43,0.488,10.0,25.0,0.400,11,18,0.611,0.605,2,8,0.250,6.0,14.0,20.0,2,2.0,0.0,3.0,10,54
4,30462,2023,4219,Aaron Gordon,NaN,PF,27.0,9,NBA,DEN,68,68.0,2055.0,429,761,0.564,60.0,173.0,0.347,369,588,0.628,0.603,191,314,0.608,164.0,282.0,446.0,203,54.0,51.0,98.0,129,1109


#### Since this data is season by season, there are duplicates. I want to combine these individual seasons into a career dataframe

In [1792]:
# Calculate career totals for various statistics
aggregated_stats = ['pts', 'ast', 'trb', 'stl', 'blk', 'g', 'mp', 'x3pa', 'x3p', 'fga', 'fg']
agg_funcs = {stat: 'sum' for stat in aggregated_stats}

# Group by player_id and aggregate statistics
careers_stats = seasons.groupby('player_id', as_index=False).agg(agg_funcs)

# Create a DataFrame for player_id
unique_player_ids = seasons['player_id'].unique()
player_id_df = pd.DataFrame({'player_id': unique_player_ids})

# Merge the player_id DataFrame with aggregated statistics
careers = player_id_df.merge(careers_stats, on='player_id', how='left')

# Map player names to careers DataFrame
names = seasons.groupby('player_id')['player'].first()
careers['player'] = careers['player_id'].map(names)

# Reorder columns
columns_order = ['player_id', 'player'] + aggregated_stats
careers = careers[columns_order]

# rename some columns
careers.rename(columns={
    'x3p' : '3ptm',
    'x3pa' : '3pta',
    'fg' : 'fgm'}, inplace=True)

In [1793]:
careers[careers['player'] == 'LeBron James']

,player_id,player,pts,ast,trb,stl,blk,g,mp,3pta,3ptm,fga,fgm
333,3463,LeBron James,38652,10420,10667.0,2186.0,1073.0,1421,54093.0,6563.0,2261.0,28044,14152


In [1794]:
# calculating CAREER per game statistics from totals divided by games
# calculating percentages from made divided by attempted
careers['ppg'] = (careers['pts'] / careers['g']).round(1)
careers['apg'] = (careers['ast'] / careers['g']).round(1)
careers['rpg'] = (careers['trb'] / careers['g']).round(1)
careers['spg'] = (careers['stl'] / careers['g']).round(1)
careers['bpg'] = (careers['blk'] / careers['g']).round(1)
careers['3pt%'] = (careers['3ptm'] / careers['3pta']).round(3)
careers['fg%'] = (careers['fgm'] / careers['fga']).round(3)
careers['mpg'] = (careers['mp'] / careers['g']).round(1)
careers.drop(columns=['mp'], inplace=True) # total minutes played is not really a statistic I need. games and mpg is enough
# if a player has never attempted a 3pt/fg, the % will be NaN, but I want to fill them with 0 instead
careers['3pt%'] = careers['3pt%'].fillna(0) 
careers['fg%'] = careers['fg%'].fillna(0)

In [1795]:
careers.head() # careers dataframe including per game stats

,player_id,player,pts,ast,trb,stl,blk,g,3pta,3ptm,fga,fgm,ppg,apg,rpg,spg,bpg,3pt%,fg%,mpg
0,5025,A.J. Green,154,22,45.0,6.0,0.0,35,105.0,44.0,125,53,4.4,0.6,1.3,0.2,0.0,0.419,0.424,9.9
1,5026,A.J. Lawson,112,4,42.0,4.0,0.0,30,50.0,20.0,88,44,3.7,0.1,1.4,0.1,0.0,0.400,0.500,7.2
2,4219,Aaron Gordon,8614,1677,4040.0,476.0,420.0,646,2168.0,706.0,6923,3265,13.3,2.6,6.3,0.7,0.7,0.326,0.472,29.0
3,4582,Aaron Holiday,2443,830,630.0,243.0,72.0,371,824.0,311.0,2157,902,6.6,2.2,1.7,0.7,0.2,0.377,0.418,17.1
4,4805,Aaron Nesmith,1152,143,493.0,90.0,48.0,171,537.0,186.0,950,402,6.7,0.8,2.9,0.5,0.3,0.346,0.423,17.9


In [1796]:
# calculating SEASON per game statistics
seasons['ppg'] = (seasons['pts'] / seasons['g']).round(1)
seasons['apg'] = (seasons['ast'] / seasons['g']).round(1)
seasons['rpg'] = (seasons['trb'] / seasons['g']).round(1)
seasons['spg'] = (seasons['stl'] / seasons['g']).round(1)
seasons['bpg'] = (seasons['blk'] / seasons['g']).round(1)

In [1797]:
seasons.head()

,seas_id,season,player_id,player,birth_year,pos,age,experience,lg,tm,g,gs,mp,fg,fga,fg_percent,x3p,x3pa,x3p_percent,x2p,x2pa,x2p_percent,e_fg_percent,ft,fta,ft_percent,orb,drb,trb,ast,stl,blk,tov,pf,pts,ppg,apg,rpg,spg,bpg
0,30458,2023,5025,A.J. Green,NaN,SG,23.0,1,NBA,MIL,35,1.0,345.0,53,125,0.424,44.0,105.0,0.419,9,20,0.450,0.600,4,4,1.000,6.0,39.0,45.0,22,6.0,0.0,9.0,31,154,4.4,0.6,1.3,0.2,0.0
1,30459,2023,5026,A.J. Lawson,NaN,SG,22.0,1,NBA,TOT,15,0.0,108.0,22,44,0.500,10.0,25.0,0.400,12,19,0.632,0.614,2,8,0.250,6.0,15.0,21.0,2,2.0,0.0,3.0,11,56,3.7,0.1,1.4,0.1,0.0
2,30460,2023,5026,A.J. Lawson,NaN,SG,22.0,1,NBA,MIN,1,0.0,2.0,1,1,1.000,0.0,0.0,NaN,1,1,1.000,1.000,0,0,NaN,0.0,1.0,1.0,0,0.0,0.0,0.0,1,2,2.0,0.0,1.0,0.0,0.0
3,30461,2023,5026,A.J. Lawson,NaN,SG,22.0,1,NBA,DAL,14,0.0,106.0,21,43,0.488,10.0,25.0,0.400,11,18,0.611,0.605,2,8,0.250,6.0,14.0,20.0,2,2.0,0.0,3.0,10,54,3.9,0.1,1.4,0.1,0.0
4,30462,2023,4219,Aaron Gordon,NaN,PF,27.0,9,NBA,DEN,68,68.0,2055.0,429,761,0.564,60.0,173.0,0.347,369,588,0.628,0.603,191,314,0.608,164.0,282.0,446.0,203,54.0,51.0,98.0,129,1109,16.3,3.0,6.6,0.8,0.8


In [1798]:
# Grouping and calculating rookie year, retire year, and total years played
group = seasons.groupby('player_id')
rookie_year = group['season'].min()
retire_year = group['season'].max()
total_years = retire_year - rookie_year + 1

# Assigning values to the careers DataFrame based on player_id
careers['first year'] = rookie_year.loc[careers['player_id']].values
careers['last year'] = retire_year.loc[careers['player_id']].values
careers['years played'] = total_years.loc[careers['player_id']].values

In [1799]:
careers.head() # careers data frame including years

,player_id,player,pts,ast,trb,stl,blk,g,3pta,3ptm,fga,fgm,ppg,apg,rpg,spg,bpg,3pt%,fg%,mpg,first year,last year,years played
0,5025,A.J. Green,154,22,45.0,6.0,0.0,35,105.0,44.0,125,53,4.4,0.6,1.3,0.2,0.0,0.419,0.424,9.9,2023,2023,1
1,5026,A.J. Lawson,112,4,42.0,4.0,0.0,30,50.0,20.0,88,44,3.7,0.1,1.4,0.1,0.0,0.400,0.500,7.2,2023,2023,1
2,4219,Aaron Gordon,8614,1677,4040.0,476.0,420.0,646,2168.0,706.0,6923,3265,13.3,2.6,6.3,0.7,0.7,0.326,0.472,29.0,2015,2023,9
3,4582,Aaron Holiday,2443,830,630.0,243.0,72.0,371,824.0,311.0,2157,902,6.6,2.2,1.7,0.7,0.2,0.377,0.418,17.1,2019,2023,5
4,4805,Aaron Nesmith,1152,143,493.0,90.0,48.0,171,537.0,186.0,950,402,6.7,0.8,2.9,0.5,0.3,0.346,0.423,17.9,2021,2023,3


#### Now I want to add some more relevant columns from other datasets (also from Sumitro Datta on kaggle)

In [1800]:
# It could be useful to know what years a player was an all star in and how many times
all_stars = pd.read_csv("All-Star Selections.csv") 
seasons['all star'] = False # create a new column indicating whether or not the player was an all star that year

for index, row in seasons.iterrows():
    player = row['player']
    season_year = row['season']
    
    # Check if the player-season combination exists in 'all_stars' DataFrame
    if ((all_stars['player'] == player) & (all_stars['season'] == season_year)).any():
        seasons.loc[index, 'all star'] = True

In [1801]:
seasons[seasons['player'] == 'Ben Simmons'] # checking that the all star column worked correctly

,seas_id,season,player_id,player,birth_year,pos,age,experience,lg,tm,g,gs,mp,fg,fga,fg_percent,x3p,x3pa,x3p_percent,x2p,x2pa,x2p_percent,e_fg_percent,ft,fta,ft_percent,orb,drb,trb,ast,stl,blk,tov,pf,pts,ppg,apg,rpg,spg,bpg,all star
32,30490,2023,4474,Ben Simmons,NaN,PG,26.0,5,NBA,BRK,42,33.0,1105.0,133,235,0.566,0.0,2.0,0.000,133,233,0.571,0.566,25,57,0.439,40.0,223.0,263.0,256,54.0,24.0,97.0,139,291,6.9,6.1,6.3,1.3,0.6,False
1540,28991,2021,4474,Ben Simmons,NaN,PG,24.0,4,NBA,PHI,58,58.0,1877.0,325,583,0.557,3.0,10.0,0.300,322,573,0.562,0.560,176,287,0.613,93.0,324.0,417.0,401,93.0,35.0,173.0,171,829,14.3,6.9,7.2,1.6,0.6,True
2241,28336,2020,4474,Ben Simmons,NaN,PG,23.0,3,NBA,PHI,57,57.0,2017.0,375,647,0.580,2.0,7.0,0.286,373,640,0.583,0.581,185,298,0.621,113.0,331.0,444.0,455,119.0,33.0,200.0,186,937,16.4,8.0,7.8,2.1,0.6,True
2893,27629,2019,4474,Ben Simmons,NaN,PG,22.0,2,NBA,PHI,79,79.0,2700.0,540,960,0.563,0.0,6.0,0.000,540,954,0.566,0.563,257,428,0.600,172.0,525.0,697.0,610,112.0,61.0,274.0,209,1337,16.9,7.7,8.8,1.4,0.8,True
3598,26962,2018,4474,Ben Simmons,NaN,PG,21.0,1,NBA,PHI,81,81.0,2732.0,544,998,0.545,0.0,11.0,0.000,544,987,0.551,0.545,191,341,0.560,145.0,514.0,659.0,661,140.0,70.0,278.0,211,1279,15.8,8.2,8.1,1.7,0.9,False


In [1802]:
# I also want to know what players made the hall of fame
hof = pd.read_csv('Player Career Info.csv') # from Sumitro Datta on Kaggle
careers = pd.merge(careers, hof[['player_id', 'hof']], how='left', on='player_id')

In [1803]:
careers[careers['player'] == 'Michael Jordan']

,player_id,player,pts,ast,trb,stl,blk,g,3pta,3ptm,fga,fgm,ppg,apg,rpg,spg,bpg,3pt%,fg%,mpg,first year,last year,years played,hof
2122,2193,Michael Jordan,32292,5633,6672.0,2514.0,893.0,1072,1778.0,581.0,24537,12192,30.1,5.3,6.2,2.3,0.8,0.327,0.497,38.3,1985,2003,19,True


In [1804]:
# Awards and award shares for mvp, dpoy could be useful
awards = pd.read_csv('Player Award Shares.csv')
seasons['MVP'] = False
seasons['DPOY'] = False
seasons['MVP share'] = 0.0
seasons['DPOY share'] = 0.0

# Calculate Award Wins
for index, row in seasons.iterrows():
    player_id = row['player_id']
    season_year = row['season']
    
    # Check if the player-season combination exists in awards DataFrame
    mask = (awards['player_id'] == player_id) & (awards['season'] == season_year)
    
    if mask.any():
        if 'nba mvp' in awards.loc[mask, 'award'].values:
            if True in awards.loc[mask & (awards['award'] == 'nba mvp'), 'winner'].values:
                seasons.loc[index, 'MVP'] = True
        if 'dpoy' in awards.loc[mask, 'award'].values:
            if True in awards.loc[mask & (awards['award'] == 'dpoy'), 'winner'].values:
                seasons.loc[index, 'DPOY'] = True


# Calculate Award Shares
for index, row in seasons.iterrows():
    player_id = row['player_id']
    season_year = row['season']    
    # Check if the player-season combination exists in awards DataFrame
    mask = (awards['player_id'] == player_id) & (awards['season'] == season_year)    
    if mask.any(): # if the combo does exist
        if 'nba mvp' in awards.loc[mask, 'award'].values:
            seasons.loc[index, 'MVP share'] = awards.loc[mask & (awards['award'] == 'nba mvp'), 'share'].values[0]           
            if 'True' in awards.loc[mask & (awards['award'] == 'nba mvp'), 'winner'].values:
                seasons.loc[index, 'MVP'] = True      
        if 'dpoy' in awards.loc[mask, 'award'].values:
            seasons.loc[index, 'DPOY share'] = awards.loc[mask & (awards['award'] == 'dpoy'), 'share'].values[0]    
            if 'True' in awards.loc[mask & (awards['award'] == 'dpoy'), 'winner'].values:
                seasons.loc[index, 'DPOY'] = True

In [1805]:
seasons.head()

,seas_id,season,player_id,player,birth_year,pos,age,experience,lg,tm,g,gs,mp,fg,fga,fg_percent,x3p,x3pa,x3p_percent,x2p,x2pa,x2p_percent,e_fg_percent,ft,fta,ft_percent,orb,drb,trb,ast,stl,blk,tov,pf,pts,ppg,apg,rpg,spg,bpg,all star,MVP,DPOY,MVP share,DPOY share
0,30458,2023,5025,A.J. Green,NaN,SG,23.0,1,NBA,MIL,35,1.0,345.0,53,125,0.424,44.0,105.0,0.419,9,20,0.450,0.600,4,4,1.000,6.0,39.0,45.0,22,6.0,0.0,9.0,31,154,4.4,0.6,1.3,0.2,0.0,False,False,False,0.0,0.0
1,30459,2023,5026,A.J. Lawson,NaN,SG,22.0,1,NBA,TOT,15,0.0,108.0,22,44,0.500,10.0,25.0,0.400,12,19,0.632,0.614,2,8,0.250,6.0,15.0,21.0,2,2.0,0.0,3.0,11,56,3.7,0.1,1.4,0.1,0.0,False,False,False,0.0,0.0
2,30460,2023,5026,A.J. Lawson,NaN,SG,22.0,1,NBA,MIN,1,0.0,2.0,1,1,1.000,0.0,0.0,NaN,1,1,1.000,1.000,0,0,NaN,0.0,1.0,1.0,0,0.0,0.0,0.0,1,2,2.0,0.0,1.0,0.0,0.0,False,False,False,0.0,0.0
3,30461,2023,5026,A.J. Lawson,NaN,SG,22.0,1,NBA,DAL,14,0.0,106.0,21,43,0.488,10.0,25.0,0.400,11,18,0.611,0.605,2,8,0.250,6.0,14.0,20.0,2,2.0,0.0,3.0,10,54,3.9,0.1,1.4,0.1,0.0,False,False,False,0.0,0.0
4,30462,2023,4219,Aaron Gordon,NaN,PF,27.0,9,NBA,DEN,68,68.0,2055.0,429,761,0.564,60.0,173.0,0.347,369,588,0.628,0.603,191,314,0.608,164.0,282.0,446.0,203,54.0,51.0,98.0,129,1109,16.3,3.0,6.6,0.8,0.8,False,False,False,0.0,0.0


#### These have all been basic NBA statistics so far... advanced statistics may be useful later in the seasons data so I am going to add some I feel are important from another one of Sumitro Datta's datasets

In [1806]:
advanced = pd.read_csv('Advanced.csv')

In [1807]:
advanced.head()

,seas_id,season,player_id,player,birth_year,pos,age,experience,lg,tm,g,mp,per,ts_percent,x3p_ar,f_tr,orb_percent,drb_percent,trb_percent,ast_percent,stl_percent,blk_percent,tov_percent,usg_percent,ows,dws,ws,ws_48,obpm,dbpm,bpm,vorp
0,30458,2023,5025,A.J. Green,NaN,SG,23.0,1,NBA,MIL,35,345.0,11.5,0.607,0.840,0.032,1.9,11.3,6.8,8.7,0.8,0.0,6.6,16.6,0.5,0.3,0.8,0.111,-0.3,-0.6,-0.9,0.1
1,30459,2023,5026,A.J. Lawson,NaN,SG,22.0,1,NBA,TOT,15,108.0,13.9,0.589,0.568,0.182,6.4,16.3,11.3,3.0,0.9,0.0,5.9,21.1,0.1,0.1,0.1,0.063,-1.5,-2.9,-4.4,-0.1
2,30460,2023,5026,A.J. Lawson,NaN,SG,22.0,1,NBA,MIN,1,2.0,34.4,1.000,0.000,0.000,0.0,54.9,27.9,0.0,0.0,0.0,0.0,21.4,0.0,0.0,0.0,0.377,-2.3,7.5,5.2,0.0
3,30461,2023,5026,A.J. Lawson,NaN,SG,22.0,1,NBA,DAL,14,106.0,13.5,0.580,0.581,0.186,6.5,15.5,11.0,3.0,0.9,0.0,6.1,21.1,0.1,0.1,0.1,0.057,-1.5,-3.1,-4.6,-0.1
4,30462,2023,4219,Aaron Gordon,NaN,PF,27.0,9,NBA,DEN,68,2055.0,19.5,0.617,0.227,0.413,9.4,15.4,12.5,14.2,1.3,2.2,9.8,21.1,4.6,2.2,6.8,0.158,2.4,-0.3,2.1,2.1


In [1808]:
seasons = pd.concat([seasons, advanced[['per', 'ts_percent', 'usg_percent', 'ows', 'dws','ws', 'obpm', 'dbpm', 'bpm', 'vorp']]], axis=1)
seasons['usg_percent'] = seasons['usg_percent'] / 100 # to make it uniform format

In [1809]:
seasons.head()

,seas_id,season,player_id,player,birth_year,pos,age,experience,lg,tm,g,gs,mp,fg,fga,fg_percent,x3p,x3pa,x3p_percent,x2p,x2pa,x2p_percent,e_fg_percent,ft,fta,ft_percent,orb,drb,trb,ast,stl,blk,tov,pf,pts,ppg,apg,rpg,spg,bpg,all star,MVP,DPOY,MVP share,DPOY share,per,ts_percent,usg_percent,ows,dws,ws,obpm,dbpm,bpm,vorp
0,30458,2023,5025,A.J. Green,NaN,SG,23.0,1,NBA,MIL,35,1.0,345.0,53,125,0.424,44.0,105.0,0.419,9,20,0.450,0.600,4,4,1.000,6.0,39.0,45.0,22,6.0,0.0,9.0,31,154,4.4,0.6,1.3,0.2,0.0,False,False,False,0.0,0.0,11.5,0.607,0.166,0.5,0.3,0.8,-0.3,-0.6,-0.9,0.1
1,30459,2023,5026,A.J. Lawson,NaN,SG,22.0,1,NBA,TOT,15,0.0,108.0,22,44,0.500,10.0,25.0,0.400,12,19,0.632,0.614,2,8,0.250,6.0,15.0,21.0,2,2.0,0.0,3.0,11,56,3.7,0.1,1.4,0.1,0.0,False,False,False,0.0,0.0,13.9,0.589,0.211,0.1,0.1,0.1,-1.5,-2.9,-4.4,-0.1
2,30460,2023,5026,A.J. Lawson,NaN,SG,22.0,1,NBA,MIN,1,0.0,2.0,1,1,1.000,0.0,0.0,NaN,1,1,1.000,1.000,0,0,NaN,0.0,1.0,1.0,0,0.0,0.0,0.0,1,2,2.0,0.0,1.0,0.0,0.0,False,False,False,0.0,0.0,34.4,1.000,0.214,0.0,0.0,0.0,-2.3,7.5,5.2,0.0
3,30461,2023,5026,A.J. Lawson,NaN,SG,22.0,1,NBA,DAL,14,0.0,106.0,21,43,0.488,10.0,25.0,0.400,11,18,0.611,0.605,2,8,0.250,6.0,14.0,20.0,2,2.0,0.0,3.0,10,54,3.9,0.1,1.4,0.1,0.0,False,False,False,0.0,0.0,13.5,0.580,0.211,0.1,0.1,0.1,-1.5,-3.1,-4.6,-0.1
4,30462,2023,4219,Aaron Gordon,NaN,PF,27.0,9,NBA,DEN,68,68.0,2055.0,429,761,0.564,60.0,173.0,0.347,369,588,0.628,0.603,191,314,0.608,164.0,282.0,446.0,203,54.0,51.0,98.0,129,1109,16.3,3.0,6.6,0.8,0.8,False,False,False,0.0,0.0,19.5,0.617,0.211,4.6,2.2,6.8,2.4,-0.3,2.1,2.1


#### A lot of these advanced statistics are measured using exact game statistics, and even possession data that I do not have, so it is difficult to translate this into a career metric. However, I can add the win share data. 

In [1810]:
# win shares an additive metric, so I can add the season win shares together for career win shares
ws = seasons.groupby('player_id')['ws'].sum()
ows = seasons.groupby('player_id')['ows'].sum()
dws = seasons.groupby('player_id')['dws'].sum()
careers = careers.merge(ows, on='player_id', how='left')
careers = careers.merge(dws, on='player_id', how='left')
careers = careers.merge(ws, on='player_id', how='left')
# total career win shares is heavily dependent on games played. a typical standardized metric is win shares per 48 minutes
careers['ws/48'] = ((careers['ws'] / (careers['g'] * careers['mpg'])) * 48).round(2)
careers['ows/48'] = ((careers['ows'] / (careers['g'] * careers['mpg'])) * 48).round(2)
careers['dws/48'] = ((careers['dws'] / (careers['g'] * careers['mpg'])) * 48).round(2)
# A few win share data is Null, so I will make them 0
careers['ws'].fillna(0, axis=0, inplace=True)

In [1811]:
careers.head()

,player_id,player,pts,ast,trb,stl,blk,g,3pta,3ptm,fga,fgm,ppg,apg,rpg,spg,bpg,3pt%,fg%,mpg,first year,last year,years played,hof,ows,dws,ws,ws/48,ows/48,dws/48
0,5025,A.J. Green,154,22,45.0,6.0,0.0,35,105.0,44.0,125,53,4.4,0.6,1.3,0.2,0.0,0.419,0.424,9.9,2023,2023,1,False,0.5,0.3,0.8,0.11,0.07,0.04
1,5026,A.J. Lawson,112,4,42.0,4.0,0.0,30,50.0,20.0,88,44,3.7,0.1,1.4,0.1,0.0,0.400,0.500,7.2,2023,2023,1,False,0.2,0.2,0.2,0.04,0.04,0.04
2,4219,Aaron Gordon,8614,1677,4040.0,476.0,420.0,646,2168.0,706.0,6923,3265,13.3,2.6,6.3,0.7,0.7,0.326,0.472,29.0,2015,2023,9,False,18.8,18.8,37.6,0.10,0.05,0.05
3,4582,Aaron Holiday,2443,830,630.0,243.0,72.0,371,824.0,311.0,2157,902,6.6,2.2,1.7,0.7,0.2,0.377,0.418,17.1,2019,2023,5,False,1.2,5.7,7.0,0.05,0.01,0.04
4,4805,Aaron Nesmith,1152,143,493.0,90.0,48.0,171,537.0,186.0,950,402,6.7,0.8,2.9,0.5,0.3,0.346,0.423,17.9,2021,2023,3,False,1.0,2.6,3.5,0.05,0.02,0.04


#### Win percentage can be an indicator of player success/value, so I am going to add it as a column to both seasons and careers data

#### Note:  I do not have information on the exact games each player played for their teams if they moved during the season, so each win percentage will be the teams total win percentage for the season, regardless if they switched teams or not

In [1812]:
teams = pd.read_csv('Team Summaries.csv') # from Sumitro Datta on Kaggle
teams['win%'] = (teams['w'] / (teams['w'] + teams['l'])).round(3) # calculating win percentage from wins divided by W+L
seasons = seasons.merge(teams[['season', 'abbreviation', 'win%']], left_on=['season', 'tm'], right_on=['season', 'abbreviation'], how='left')
seasons.drop(columns=['abbreviation'], inplace=True) # drop the duplicate team column

In [1813]:
seasons.head()

,seas_id,season,player_id,player,birth_year,pos,age,experience,lg,tm,g,gs,mp,fg,fga,fg_percent,x3p,x3pa,x3p_percent,x2p,x2pa,x2p_percent,e_fg_percent,ft,fta,ft_percent,orb,drb,trb,ast,stl,blk,tov,pf,pts,ppg,apg,rpg,spg,bpg,all star,MVP,DPOY,MVP share,DPOY share,per,ts_percent,usg_percent,ows,dws,ws,obpm,dbpm,bpm,vorp,win%
0,30458,2023,5025,A.J. Green,NaN,SG,23.0,1,NBA,MIL,35,1.0,345.0,53,125,0.424,44.0,105.0,0.419,9,20,0.450,0.600,4,4,1.000,6.0,39.0,45.0,22,6.0,0.0,9.0,31,154,4.4,0.6,1.3,0.2,0.0,False,False,False,0.0,0.0,11.5,0.607,0.166,0.5,0.3,0.8,-0.3,-0.6,-0.9,0.1,0.707
1,30459,2023,5026,A.J. Lawson,NaN,SG,22.0,1,NBA,TOT,15,0.0,108.0,22,44,0.500,10.0,25.0,0.400,12,19,0.632,0.614,2,8,0.250,6.0,15.0,21.0,2,2.0,0.0,3.0,11,56,3.7,0.1,1.4,0.1,0.0,False,False,False,0.0,0.0,13.9,0.589,0.211,0.1,0.1,0.1,-1.5,-2.9,-4.4,-0.1,NaN
2,30460,2023,5026,A.J. Lawson,NaN,SG,22.0,1,NBA,MIN,1,0.0,2.0,1,1,1.000,0.0,0.0,NaN,1,1,1.000,1.000,0,0,NaN,0.0,1.0,1.0,0,0.0,0.0,0.0,1,2,2.0,0.0,1.0,0.0,0.0,False,False,False,0.0,0.0,34.4,1.000,0.214,0.0,0.0,0.0,-2.3,7.5,5.2,0.0,0.512
3,30461,2023,5026,A.J. Lawson,NaN,SG,22.0,1,NBA,DAL,14,0.0,106.0,21,43,0.488,10.0,25.0,0.400,11,18,0.611,0.605,2,8,0.250,6.0,14.0,20.0,2,2.0,0.0,3.0,10,54,3.9,0.1,1.4,0.1,0.0,False,False,False,0.0,0.0,13.5,0.580,0.211,0.1,0.1,0.1,-1.5,-3.1,-4.6,-0.1,0.463
4,30462,2023,4219,Aaron Gordon,NaN,PF,27.0,9,NBA,DEN,68,68.0,2055.0,429,761,0.564,60.0,173.0,0.347,369,588,0.628,0.603,191,314,0.608,164.0,282.0,446.0,203,54.0,51.0,98.0,129,1109,16.3,3.0,6.6,0.8,0.8,False,False,False,0.0,0.0,19.5,0.617,0.211,4.6,2.2,6.8,2.4,-0.3,2.1,2.1,0.646


#### Adding career win percentage

In [1814]:
seasons['wins'] = seasons['g'] * (seasons['win%'])
total_wins = seasons.groupby('player_id')['wins'].sum().round(0)
careers = careers.merge(total_wins, on='player_id', how='left')
careers['win%'] = (careers['wins'] / careers['g']).round(2)
careers.drop(columns=['wins'], inplace=True) # I don't think I need wins anymore when I have games and win%

In [1815]:
careers.head()

,player_id,player,pts,ast,trb,stl,blk,g,3pta,3ptm,fga,fgm,ppg,apg,rpg,spg,bpg,3pt%,fg%,mpg,first year,last year,years played,hof,ows,dws,ws,ws/48,ows/48,dws/48,win%
0,5025,A.J. Green,154,22,45.0,6.0,0.0,35,105.0,44.0,125,53,4.4,0.6,1.3,0.2,0.0,0.419,0.424,9.9,2023,2023,1,False,0.5,0.3,0.8,0.11,0.07,0.04,0.71
1,5026,A.J. Lawson,112,4,42.0,4.0,0.0,30,50.0,20.0,88,44,3.7,0.1,1.4,0.1,0.0,0.400,0.500,7.2,2023,2023,1,False,0.2,0.2,0.2,0.04,0.04,0.04,0.23
2,4219,Aaron Gordon,8614,1677,4040.0,476.0,420.0,646,2168.0,706.0,6923,3265,13.3,2.6,6.3,0.7,0.7,0.326,0.472,29.0,2015,2023,9,False,18.8,18.8,37.6,0.10,0.05,0.05,0.42
3,4582,Aaron Holiday,2443,830,630.0,243.0,72.0,371,824.0,311.0,2157,902,6.6,2.2,1.7,0.7,0.2,0.377,0.418,17.1,2019,2023,5,False,1.2,5.7,7.0,0.05,0.01,0.04,0.45
4,4805,Aaron Nesmith,1152,143,493.0,90.0,48.0,171,537.0,186.0,950,402,6.7,0.8,2.9,0.5,0.3,0.346,0.423,17.9,2021,2023,3,False,1.0,2.6,3.5,0.05,0.02,0.04,0.51


#### Since all stars are selected by conference, I need to add a column for the players conference

#### There is not a set rule for players that switch conferences during the season, so I will just make their conference match the first team they played for that season. This row is always found under the total row (for example, Kevin Durant played for 2 teams in 2023, so he has 3 rows: a total row, Brooklyn, then Phoenix). This row is indicated in the 'tm' column with 'TOT'

In [1816]:
# There were other leagues like the ABA before the NBA became dominant. I will only be using NBA data for consistency
seasons = seasons[seasons['lg'] == 'NBA']
# list of team names
seasons['tm'].unique()

array(['MIL', 'TOT', 'MIN', 'DAL', 'DEN', 'ATL', 'IND', 'OKC', 'ORL',
       'BOS', 'DET', 'CHI', 'SAC', 'SAS', 'BRK', 'HOU', 'LAC', 'GSW',
       'POR', 'LAL', 'WAS', 'MIA', 'PHO', 'MEM', 'NOP', 'CHO', 'NYK',
       'CLE', 'TOR', 'UTA', 'PHI', 'CHA', 'NOH', 'NJN', 'SEA', 'NOK',
       'CHH', 'VAN', 'WSB', 'KCK', 'SDC', 'NOJ', 'BUF', 'NYN', 'KCO',
       'CAP', 'BAL', 'CIN', 'SFW', 'SDR', 'STL', 'SYR', 'CHZ', 'PHW',
       'CHP', 'MNL', 'FTW', 'ROC', 'BLB', 'MLH', 'INO', 'WSC', 'TRI',
       'DNN', 'WAT', 'CHS', 'STB', 'AND', 'SHE'], dtype=object)

In [1817]:
east = ['MIL', 'ATL', 'IND', 'ORL', 'BOS', 'DET', 'CHI', 'BRK', 'WAS', 'MIA', 'CHO', 'NYK', 'CLE', 'TOR', 'PHI', 'CHA','NJN',
        'CHH', 'VAN', 'WSB', 'BUF', 'NYK', 'CAP', 'BAL', 'CIN', 'STL', 'SYR', 'CHZ', 'PHW', 'CHP', 'FTW', 'ROC', 'BLB', 'MLH', 
        'INO', 'WSC', 'TRI', 'WAT', 'CHS', 'STB', 'AND', 'SHE']
seasons['conference'] = 'west'
seasons.loc[seasons['tm'].isin(east), 'conference'] = 'east'

# now I have to deal with the total ('TOT') rows mentioned above
teams = seasons.groupby(['season', 'player'])['tm']
tot_rows = seasons['tm'] == 'TOT' # gather the total rows for players that switched teams
seasons.loc[tot_rows, 'conference'] = seasons['conference'].shift(-1) # change the conference to the conference of the row below
# as explained above, the row below the 'TOT' row is the first team the player played for that season

In [1818]:
seasons[seasons['player'] == 'Kevin Durant'].head() 
# as planned, the conference for KD's recent 'TOT' row is East because he started his season with the nets

,seas_id,season,player_id,player,birth_year,pos,age,experience,lg,tm,g,gs,mp,fg,fga,fg_percent,x3p,x3pa,x3p_percent,x2p,x2pa,x2p_percent,e_fg_percent,ft,fta,ft_percent,orb,drb,trb,ast,stl,blk,tov,pf,pts,ppg,apg,rpg,spg,bpg,all star,MVP,DPOY,MVP share,DPOY share,per,ts_percent,usg_percent,ows,dws,ws,obpm,dbpm,bpm,vorp,win%,wins,conference
381,30839,2023,3770,Kevin Durant,NaN,PF-SF,34.0,15,NBA,TOT,47,47.0,1672.0,483,862,0.560,93.0,230.0,0.404,390,632,0.617,0.614,307,334,0.919,17.0,296.0,313.0,235,34.0,67.0,156.0,99,1366,29.1,5.0,6.7,0.7,1.4,True,False,False,0.000,0.0,25.9,0.677,0.307,4.7,2.1,6.8,6.0,1.2,7.1,3.9,NaN,NaN,east
382,30840,2023,3770,Kevin Durant,NaN,PF,34.0,15,NBA,BRK,39,39.0,1403.0,410,734,0.559,71.0,189.0,0.376,339,545,0.622,0.607,267,286,0.934,14.0,248.0,262.0,207,32.0,57.0,136.0,92,1158,29.7,5.3,6.7,0.8,1.5,True,False,False,0.000,0.0,26.2,0.673,0.315,3.9,1.7,5.7,5.8,1.2,7.0,3.2,0.549,21.411,east
383,30841,2023,3770,Kevin Durant,NaN,SF,34.0,15,NBA,PHO,8,8.0,269.0,73,128,0.570,22.0,41.0,0.537,51,87,0.586,0.656,40,48,0.833,3.0,48.0,51.0,28,2.0,10.0,20.0,7,208,26.0,3.5,6.4,0.2,1.2,True,False,False,0.000,0.0,24.3,0.697,0.268,0.8,0.3,1.1,7.1,0.8,7.9,0.7,0.549,4.392,west
1142,30110,2022,3770,Kevin Durant,NaN,PF,33.0,14,NBA,BRK,55,55.0,2047.0,578,1115,0.518,115.0,300.0,0.383,463,815,0.568,0.570,372,409,0.910,29.0,378.0,407.0,351,48.0,52.0,191.0,113,1643,29.9,6.4,7.4,0.9,0.9,True,False,False,0.001,0.0,25.6,0.634,0.312,6.4,2.0,8.4,6.4,0.7,7.2,4.8,0.537,29.535,east
1888,29339,2021,3770,Kevin Durant,NaN,PF,32.0,13,NBA,BRK,35,32.0,1157.0,324,603,0.537,85.0,189.0,0.450,239,414,0.577,0.608,210,238,0.882,13.0,234.0,247.0,195,25.0,45.0,120.0,70,943,26.9,5.6,7.1,0.7,1.3,True,False,False,0.000,0.0,26.4,0.666,0.312,3.7,1.2,5.0,6.4,0.8,7.2,2.7,0.667,23.345,east


#### Now I am going to combine the players that played for multiple teams in one season, so each player only has one row per season

#### Note: I am filling win% for players who played for multiple teams in a season, who's total stats are denoted with 'TOT' under the tm column, with the league average win percentage. This is because their team-specific win percentage is not representative or as valuable due to playing for multiple teams. Then, I will delete the team-specific rows for the players that switched midseason, so there will only be the total stats

In [1819]:
avg_win_pct = seasons['win%'].mean() # should be close to 50%
seasons.loc[seasons['tm'] == 'TOT', 'win%'] = avg_win_pct # fill average win percentage for TOT rows
seasons = seasons.drop_duplicates(subset=['player_id', 'season'], keep='first') # TOT is the first row for all the multiple team players, so only keep TOT

In [1820]:
duplicates = seasons.duplicated(subset=['player_id', 'season'])
total_duplicates = duplicates.sum()
total_duplicates # there are no more player + season duplicates

0

#### Now that each player has one row per season, I will use the same strategy from earlier to do career all-stars, awards, and award shares

In [1821]:
# career all star appearances
careers['all stars'] = 0 
for index, row in seasons.iterrows():
    player = row['player']
    player_id = row['player_id']
    season_year = row['season']
    if ((all_stars['player'] == player) & (all_stars['season'] == season_year)).any():
        careers.loc[careers['player_id'] == player_id, 'all stars'] += 1

In [1822]:
# career award totals
mvps = seasons.groupby(['player_id'])['MVP'].sum().reset_index()
for index, row in mvps.iterrows():
    player_id = row['player_id']
    mvp_count = row['MVP']
    careers.loc[(careers['player_id'] == player_id), 'MVPs'] = mvp_count

dpoys = seasons.groupby(['player_id'])['DPOY'].sum().reset_index()
for index, row in dpoys.iterrows():
    player_id = row['player_id']
    mvp_count = row['DPOY']
    careers.loc[(careers['player_id'] == player_id), 'DPOYs'] = mvp_count

In [1823]:
# career award shares
mvp_shares = seasons.groupby('player_id')['MVP share'].sum()
careers = careers.merge(mvp_shares, on='player_id', how='left')
dpoy_shares = seasons.groupby('player_id')['DPOY share'].sum()
careers = careers.merge(dpoy_shares, on='player_id', how='left')

In [1824]:
careers[careers['player'] == 'LeBron James']

,player_id,player,pts,ast,trb,stl,blk,g,3pta,3ptm,fga,fgm,ppg,apg,rpg,spg,bpg,3pt%,fg%,mpg,first year,last year,years played,hof,ows,dws,ws,ws/48,ows/48,dws/48,win%,all stars,MVPs,DPOYs,MVP share,DPOY share
333,3463,LeBron James,38652,10420,10667.0,2186.0,1073.0,1421,6563.0,2261.0,28044,14152,27.2,7.3,7.5,1.5,0.8,0.345,0.505,38.1,2004,2023,20,False,177.1,78.1,255.3,0.23,0.16,0.07,0.63,19,4.0,0.0,8.818,0.886


#### Before I do anything else, I want to clean up the seasons dataset a little more

In [1825]:
# I was getting a lot of warnings with this block and it suggested to make a copy
seasons_copy = seasons.copy()

# Calculate minutes per game (mpg) on the copy
seasons_copy['mpg'] = (seasons_copy['mp'] / seasons_copy['g']).round(1)

# Calculate ws/48, ows/48, and dws/48 using .loc on the copy
seasons_copy.loc[:, 'ws/48'] = ((seasons_copy['ws'] / (seasons_copy['g'] * seasons_copy['mpg'])) * 48).round(2)
seasons_copy.loc[:, 'ows/48'] = ((seasons_copy['ows'] / (seasons_copy['g'] * seasons_copy['mpg'])) * 48).round(2)
seasons_copy.loc[:, 'dws/48'] = ((seasons_copy['dws'] / (seasons_copy['g'] * seasons_copy['mpg'])) * 48).round(2)

# Drop unnecessary columns using .loc on the copy
columns_to_drop = ['seas_id', 'birth_year', 'lg', 'e_fg_percent', 'orb', 'drb', 'pf', 'wins']
seasons_copy.drop(columns=columns_to_drop, axis=1, inplace=True)

In [1826]:
# I want to change the names of some columns now to be more clear and also consistent with my careers data
seasons_copy.rename(columns={
    'g' : 'games',
    'gs' : 'starts',
    'tm' : 'team',
    'fg_percent' : 'fg%',
    'x3p' : '3pm',
    'x3pa' : '3pa',
    'x3p_percent' : '3p%',
    'x2p' : '2pm',
    'x2pa' : '2pa',
    'x2p_percent' : '2p%',
    'ts_percent' : 'ts%',
    'usg_percent' : 'usg%',
    'ft_percent' : 'ft%',
}, inplace=True)

In [1827]:
seasons_copy.head()

,season,player_id,player,pos,age,experience,team,games,starts,mp,fg,fga,fg%,3pm,3pa,3p%,2pm,2pa,2p%,ft,fta,ft%,trb,ast,stl,blk,tov,pts,ppg,apg,rpg,spg,bpg,all star,MVP,DPOY,MVP share,DPOY share,per,ts%,usg%,ows,dws,ws,obpm,dbpm,bpm,vorp,win%,conference,mpg,ws/48,ows/48,dws/48
0,2023,5025,A.J. Green,SG,23.0,1,MIL,35,1.0,345.0,53,125,0.424,44.0,105.0,0.419,9,20,0.450,4,4,1.000,45.0,22,6.0,0.0,9.0,154,4.4,0.6,1.3,0.2,0.0,False,False,False,0.0,0.0,11.5,0.607,0.166,0.5,0.3,0.8,-0.3,-0.6,-0.9,0.1,0.707000,east,9.9,0.11,0.07,0.04
1,2023,5026,A.J. Lawson,SG,22.0,1,TOT,15,0.0,108.0,22,44,0.500,10.0,25.0,0.400,12,19,0.632,2,8,0.250,21.0,2,2.0,0.0,3.0,56,3.7,0.1,1.4,0.1,0.0,False,False,False,0.0,0.0,13.9,0.589,0.211,0.1,0.1,0.1,-1.5,-2.9,-4.4,-0.1,0.492212,west,7.2,0.04,0.04,0.04
4,2023,4219,Aaron Gordon,PF,27.0,9,DEN,68,68.0,2055.0,429,761,0.564,60.0,173.0,0.347,369,588,0.628,191,314,0.608,446.0,203,54.0,51.0,98.0,1109,16.3,3.0,6.6,0.8,0.8,False,False,False,0.0,0.0,19.5,0.617,0.211,4.6,2.2,6.8,2.4,-0.3,2.1,2.1,0.646000,west,30.2,0.16,0.11,0.05
5,2023,4582,Aaron Holiday,PG,26.0,5,ATL,63,6.0,845.0,92,220,0.418,36.0,88.0,0.409,56,132,0.424,27,32,0.844,74.0,89,37.0,12.0,36.0,247,3.9,1.4,1.2,0.6,0.2,False,False,False,0.0,0.0,9.4,0.528,0.134,0.3,0.6,0.9,-2.9,0.9,-2.0,0.0,0.500000,east,13.4,0.05,0.02,0.03
6,2023,4805,Aaron Nesmith,SF,23.0,3,IND,73,60.0,1816.0,252,590,0.427,115.0,314.0,0.366,137,276,0.496,119,142,0.838,277.0,98,55.0,34.0,75.0,738,10.1,1.3,3.8,0.8,0.5,False,False,False,0.0,0.0,10.5,0.566,0.168,0.9,1.1,2.0,-2.3,-0.7,-3.0,-0.4,0.427000,east,24.9,0.05,0.02,0.03


#### If there is still null data at this point after my modifications and deletions, it means the NBA had not began recorded that statistic (Ex: 3pt and blocks)
The historical data still has value in certain aspects, but I should create a new modern data set for when I am working with the data that doesn't reach all the way back.

In [1828]:
modern_seasons = seasons_copy.copy()
modern_seasons.dropna(axis=0, inplace=True)

In [1829]:
modern_seasons.info() # this cut down a lot of the older entries

<class 'pandas.core.frame.DataFrame'>
Index: 15323 entries, 0 to 23121
Data columns (total 54 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   season      15323 non-null  int64  
 1   player_id   15323 non-null  int64  
 2   player      15323 non-null  object 
 3   pos         15323 non-null  object 
 4   age         15323 non-null  float64
 5   experience  15323 non-null  int64  
 6   team        15323 non-null  object 
 7   games       15323 non-null  int64  
 8   starts      15323 non-null  float64
 9   mp          15323 non-null  float64
 10  fg          15323 non-null  int64  
 11  fga         15323 non-null  int64  
 12  fg%         15323 non-null  float64
 13  3pm         15323 non-null  float64
 14  3pa         15323 non-null  float64
 15  3p%         15323 non-null  float64
 16  2pm         15323 non-null  int64  
 17  2pa         15323 non-null  int64  
 18  2p%         15323 non-null  float64
 19  ft          15323 non-null  in

#### Lastly, I am going to add the most positions to the careers data

In [1830]:
# since some players switched positions over multiple seasons, I will find the most common position for each player and make that their career position
positions = seasons_copy.groupby('player_id')['pos'].apply(lambda x: x.mode().iloc[0]).reset_index()
careers = careers.merge(positions, on='player_id', how='left')

In [1831]:
careers[careers['player'] == 'LeBron James']

,player_id,player,pts,ast,trb,stl,blk,g,3pta,3ptm,fga,fgm,ppg,apg,rpg,spg,bpg,3pt%,fg%,mpg,first year,last year,years played,hof,ows,dws,ws,ws/48,ows/48,dws/48,win%,all stars,MVPs,DPOYs,MVP share,DPOY share,pos
333,3463,LeBron James,38652,10420,10667.0,2186.0,1073.0,1421,6563.0,2261.0,28044,14152,27.2,7.3,7.5,1.5,0.8,0.345,0.505,38.1,2004,2023,20,False,177.1,78.1,255.3,0.23,0.16,0.07,0.63,19,4.0,0.0,8.818,0.886,SF


#### I will keep the careers data as is because there are important players whos careers overlapped into modern NBA, so it would be difficult to express their impact if I cut into their career data. However, I may experiment with using all the data vs. more recent data to make predictions

In [1832]:
# create csv files to use
seasons_copy.to_csv('seasons.csv')
modern_seasons.to_csv('modern_seasons.csv')
careers.to_csv('careers.csv')